In [ ]:
import pandas as pd
import tensorflow as tf

# 下载csv的数据
train_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

'''
其中我们 columns 变量定义了全部的列，
同时我们定义了 categorical 和 numeric 两个数据列，
分别代表离散值以及连续值，这便于我们后面的编码处理。
'''
columns = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

categorical = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric = ['age', 'fare']

train_df = pd.read_csv(train_file, names=columns)

'''
然后我们进行测试集与训练集的划分：
'''
# 使用 train_df.sample() 方法来按照 0.2 的采样率进行随机采样，从来获取我们的测试集
test_df = train_df.sample(frac=0.2)
# 将原来集合中的测试集去掉，得到剩下的训练集
train_df = train_df[~train_df.index.isin(test_df.index)]

# 采用 pop 的方式来获取到标签值。这里我们按照传统的8：2的比例进行分割。
train_df_y = train_df.pop('survived')
test_df_y = test_df.pop('survived')

# 输出：502, 125
print(len(train_df), len(test_df))

'''
然后我们就进行特征列的构造，在这里我们将特征列分为连续值与离散值，
对于离散值，我们进行了独热编码的处理，因为独热编码在训练的过程中较大的优势：

在这里我们使用到了三个非常重要的函数：

tf.feature_column.numeric_column，将某一列作为连续值视作数据特征；
tf.feature_column.indicator_column，将某一离散列作为独热编码视作数据列；
tf.feature_column.categorical_column_with_vocabulary_list，
        接收第一个参数为某一离散特征列，
        第二个参数为该离散特征列的所有取值情况，将一个普通列视作离散数据列（未经过独热编码）。
'''
def transfer2one_hot(feature, feature_vocab):
  return tf.feature_column.indicator_column(tf.feature_column.categorical_column_with_vocabulary_list(feature, feature_vocab))

features = []
for feature in categorical:
  features.append(
      transfer2one_hot(feature, train_df[feature].unique())
    )

for feature in numeric:
  features.append(
      tf.feature_column.numeric_column(feature, dtype=tf.float32)
    )

'''
然后我们便可以定义我们的输入函数
在这里我们将输入函数分为训练时的输入函数以及测试时的输入函数

在输入函数之中，我们使用参数中的 features 和 labels 创建了一个数据集，并且将其返回。
并且对于训练集，我们采用了数据增强，也就是随机化以及重复操作，
在这里，30000 就是我们要训练的 Steps 数量（我们所使用的 batch_size 的数量，这里是8）。

最后因为输入函数是一个函数，因此我们需要定义一个 lambda 函数来返回一个输入函数，
否则输入函数将直接返回一个数据集，这是我们不希望看到的。
'''
def train_input_fn(features, labels, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels)) 
    dataset = dataset.shuffle(len(train_df))
    dataset = dataset.repeat(30000)   
    return dataset.batch(batch_size)
def test_input_fn(features, labels, batch_size=8):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))    
    return dataset.batch(batch_size)

train_fn = lambda: train_input_fn(train_df, train_df_y)
test_fn = lambda: test_input_fn(test_df, test_df_y)

'''
在这里我们定义了一个 Estimator 内置的分类器，它的三个参数如下：

feature_columns，特征列，就是我们刚刚处理过的特征列，包含连序列与离散列；
hidden_units，隐藏单元的数量，在这里我们定义了三层；
n_classes，输出的种类，这里我们只有生存和不能生存两类。
'''
# 再者我们就可以载入一个内置的模型：
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[50, 30, 20],
    n_classes=2)

# 最后我们便可以进行模型的训练与测试，具体的实现细节非常简单
classifier.train(input_fn=train_fn, steps=30000)

test_result = classifier.evaluate(input_fn=test_fn)
print(test_result)
